In [1]:
!pip install twython

In [0]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import twython

In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [7]:
%ls

Final_Salty.csv  sample_data/


In [0]:
#Using the Hackernews dataset

df = pd.read_csv('/content/Final_Salty.csv')

In [9]:
#SaltyScore created using Vader, highest salt on top

df.head()

,Unnamed: 0,Comment_ID,Comment,UserName,StoryId,Deleted,SaltyScore
0,1118,23334754,Fuck em,rStar,23331287,False,0.778
1,120,23338465,"Not great, not terrible",laretluval,23334339,False,0.731
2,1353,23320362,Risk aversion and capital?,eximius,23319848,False,0.714
3,1265,23335878,AMP is a terrifying solution to an awful problem.,tobyhinloopen,23322730,False,0.599
4,2595,23329336,launch aborted :(,tosh,23322948,False,0.592


In [0]:
#Use vader to create a column for classification

def analyze_sentiment_vader_lexicon(review, threshold=0.1,
                                    verbose=False):
  # analyze the sentiment for review
  analyzer = SentimentIntensityAnalyzer()
  scores = analyzer.polarity_scores(review)
  # get aggregate scores and final sentiment
  agg_score = scores['compound']
  if agg_score >= threshold:
    final_sentiment = 'positive'
  elif agg_score <= -threshold:
    final_sentiment = 'negative'
  else:
    final_sentiment = 'neutral'

  if verbose:
    # display detailed sentiment statistics
    positive = str(round(scores['pos'], 2)*100)+'%'
    final = round(agg_score, 2)
    negative = str(round(scores['neg'], 2)*100)+'%'
    neutral = str(round(scores['neu'], 2)*100)+'%'
    sentiment_frame = pd.DataFrame([[final_sentiment, final, positive,
                                      negative, neutral]], columns=pd.MultiIndex(levels=[['SENTIMENT STATS:'],['Predicted Sentiment', 'Polarity Score','Positive', 'Negative', 'Neutral']],codes=[[0,0,0,0,0],[0,1,2,3,4]]))
    print(sentiment_frame)
  return final_sentiment

In [0]:
df['sentiment'] = df['Comment'].apply(analyze_sentiment_vader_lexicon)

In [12]:
df.head()

,Unnamed: 0,Comment_ID,Comment,UserName,StoryId,Deleted,SaltyScore,sentiment
0,1118,23334754,Fuck em,rStar,23331287,False,0.778,negative
1,120,23338465,"Not great, not terrible",laretluval,23334339,False,0.731,negative
2,1353,23320362,Risk aversion and capital?,eximius,23319848,False,0.714,negative
3,1265,23335878,AMP is a terrifying solution to an awful problem.,tobyhinloopen,23322730,False,0.599,negative
4,2595,23329336,launch aborted :(,tosh,23322948,False,0.592,negative


In [13]:
#may need to adjust threshold

df['sentiment'].value_counts()

positive    1765
negative     729
neutral      568
Name: sentiment, dtype: int64

In [0]:
counter = Counter(df['sentiment'].tolist())
sent_rank = {i[0]: idx for idx, i in enumerate(counter.most_common(3))}
df = df[df['sentiment'].map(lambda x: x in sent_rank)]

In [22]:
sent_rank

{'negative': 1, 'neutral': 2, 'positive': 0}

In [0]:
comment_list = df['Comment'].tolist()
sent_list = [sent_rank[i] for i in df['sentiment'].tolist()]
sent_list = np.array(sent_list)

In [24]:
sent_list

array([1, 1, 1, ..., 0, 0, 0])

In [0]:
count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(comment_list)

In [26]:
x_train_counts.shape

(3062, 15376)

In [0]:
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

In [28]:
x_train_tfidf.shape

(3062, 15376)

In [29]:
x_train_tfidf.shape

(3062, 15376)

In [30]:
sent_list.shape, x_train_tfidf.shape

((3062,), (3062, 15376))

In [0]:
#x_train_tfidf = x_train_tfidf.reshape(x_train_tfidf.shape[1:])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x_train_tfidf, sent_list, test_size=0.3, random_state=0)

In [32]:
df.shape

(3062, 8)

In [0]:
clf = MultinomialNB().fit(X_train, y_train)

In [0]:
y_score = clf.predict(X_test)

In [0]:
n_right = 0
for i in range(len(y_score)):
  if y_score[i] == y_test[i]:
    n_right += 1

In [36]:
#Threshold .1 is the best accuracy so far.
#Classes imbalanced

print("Accuracy: %.2f%%" % ((n_right/float(len(y_test)) * 100)))

Accuracy: 57.78%


In [0]:
df_neg = df[df['sentiment']=='negative'].sample(500)


In [0]:
df_pos = df[df['sentiment']=='positive'].sample(500)

In [0]:
df_neu = df[df['sentiment']=='neutral'].sample(500)

In [0]:
frames = [df_pos, df_neg, df_neu]

df_samp = pd.concat(frames)

In [41]:
#balanced classes
df_samp

,Unnamed: 0,Comment_ID,Comment,UserName,StoryId,Deleted,SaltyScore,sentiment
1677,277,23334233,Counter-point: while in many situations this i...,mehrdadn,23333891,False,0.014,positive
1000,2058,23319658,I have colleague who still write desktop appli...,jasonlhy,23319546,False,0.066,positive
1978,2542,23318967,Amazon is buying 100K custom delivery vans so ...,wmf,23318131,False,0.000,positive
601,2746,23323213,This also seems to be missing any coursework o...,mattkrause,23321426,False,0.104,positive
1246,2448,23322903,Free speech is not just an American constituti...,falcolas,23322112,False,0.049,positive
...,...,...,...,...,...,...,...,...
2730,1928,23341131,Feedly.,lazlee,23340031,False,0.000,neutral
2187,2309,23324996,Do you need audio?,coherentpony,23322117,False,0.000,neutral
1611,2763,23333535,"&gt; Kate Klonick, assistant legal professor a...",samizdis,23336298,False,0.023,neutral
2186,1908,23340649,Push to Kindle [1]. There are extensions for b...,dwdz,23340031,False,0.000,neutral


In [0]:
#analyze new df
df_samp['sentiment'] = df_samp['Comment'].apply(analyze_sentiment_vader_lexicon)

In [0]:
counter = Counter(df_samp['sentiment'].tolist())
sent_rank = {i[0]: idx for idx, i in enumerate(counter.most_common(3))}
df_samp = df_samp[df_samp['sentiment'].map(lambda x: x in sent_rank)]

In [0]:
comment_list = df_samp['Comment'].tolist()
sent_list = [sent_rank[i] for i in df_samp['sentiment'].tolist()]
sent_list = np.array(sent_list)

In [0]:
count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(comment_list)

In [0]:
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x_train_tfidf, sent_list, test_size=0.3, random_state=0)

In [0]:
clf = MultinomialNB().fit(X_train, y_train)

In [0]:
y_score = clf.predict(X_test)

In [0]:
n_right = 0
for i in range(len(y_score)):
  if y_score[i] == y_test[i]:
    n_right += 1

In [51]:
#That didn't help at all

print("Accuracy: %.2f%%" % ((n_right/float(len(y_test)) * 100)))

Accuracy: 54.44%


In [0]:
#Convert Salty to int for classification
df['SaltyScore'] = df['SaltyScore'] *100
df['SaltyScore'] = df['SaltyScore'].astype(int)

In [0]:
#going to bin salty score values

cut_labels = ['pos', 'neu', 'neg']
cut_bins = [0, 33, 67, 100]
df['bin'] = pd.cut(df['SaltyScore'], bins=cut_bins, labels=cut_labels)

In [54]:
df

,Unnamed: 0,Comment_ID,Comment,UserName,StoryId,Deleted,SaltyScore,sentiment,bin
0,1118,23334754,Fuck em,rStar,23331287,False,77,negative,neg
1,120,23338465,"Not great, not terrible",laretluval,23334339,False,73,negative,neg
2,1353,23320362,Risk aversion and capital?,eximius,23319848,False,71,negative,neg
3,1265,23335878,AMP is a terrifying solution to an awful problem.,tobyhinloopen,23322730,False,59,negative,neu
4,2595,23329336,launch aborted :(,tosh,23322948,False,59,negative,neu
...,...,...,...,...,...,...,...,...,...
3057,1722,23338836,I’m strongly thinking of just migrating to Ubu...,samgranieri,23336255,False,0,positive,NaN
3058,1720,23337471,Proper ARM SBCL with threads yay,ivan4th,23336255,False,0,positive,NaN
3059,1719,23336865,Finally! Does this mean ArchLinuxARM (ALARM) w...,qalmakka,23336255,False,0,positive,NaN
3060,1718,23338715,Is there a noticeable performance benefit with...,mgamache,23336255,False,0,positive,NaN


In [0]:
df['bin'] = df['bin'].fillna('neg')

In [0]:
df['bin'] = df['Comment'].apply(analyze_sentiment_vader_lexicon)

In [0]:
#df = pd.DataFrame.drop(df, columns=['sentiment','SaltyScore','Deleted', 'StoryId', 'Unnamed: 0', 'Comment_ID'])
df = pd.DataFrame.drop(df, columns=['SaltyScore','Deleted', 'StoryId', 'Unnamed: 0', 'Comment_ID'])

In [58]:
df

,Comment,UserName,sentiment,bin
0,Fuck em,rStar,negative,negative
1,"Not great, not terrible",laretluval,negative,negative
2,Risk aversion and capital?,eximius,negative,negative
3,AMP is a terrifying solution to an awful problem.,tobyhinloopen,negative,negative
4,launch aborted :(,tosh,negative,negative
...,...,...,...,...
3057,I’m strongly thinking of just migrating to Ubu...,samgranieri,positive,positive
3058,Proper ARM SBCL with threads yay,ivan4th,positive,positive
3059,Finally! Does this mean ArchLinuxARM (ALARM) w...,qalmakka,positive,positive
3060,Is there a noticeable performance benefit with...,mgamache,positive,positive


In [0]:
counter = Counter(df['bin'].tolist())
salt_rank = {i[0]: idx for idx, i in enumerate(counter.most_common(3))}
df = df[df['bin'].map(lambda x: x in salt_rank)]

In [0]:
comment_list = df['bin'].tolist()
salt_list = [salt_rank[i] for i in df['bin'].tolist()]
salt_list = np.array(salt_list)

In [0]:
count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(comment_list)

In [0]:
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x_train_tfidf, salt_list, test_size=0.3, random_state=0)

In [0]:
clf = MultinomialNB().fit(X_train, y_train)

In [0]:
y_score = clf.predict(X_test)

In [0]:
n_right = 0
for i in range(len(y_score)):
  if y_score[i] == y_test[i]:
    n_right += 1

In [67]:
#probably overfit

print("Accuracy: %.2f%%" % ((n_right/float(len(y_test)) * 100)))

Accuracy: 100.00%
